![Data Engineering](img/header2.JPG "Data Engineering")

# <span style="color: darkblue">Voter Turnout in Presidential Elections</span>

We will use this notebook and project to explore, visualize, and analyze presidential election participation data, otherwise known as "voter turnout". Elections are voluntary in the United States and understanding the spatial characteristics of voter turnout is very important to understand how policy decisions are made based on who participated in the election process. 

We will begin with <span style="color:purple">Data Engineering</span>: Broadly described as the actions taken to make data useful for analysis. Whether it’s removing records with erroneous data, reformatting the structure of a table to better conduct analysis, or several other actions that may help you prepare data, Data Engineering is an important part of every analysis. 

We will start by downloading and preparing US presidential election [data from MIT's Election Data and Science Lab](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ), including handling missing values, reformatting data types, and restructuring the format of a table. Using GIS, Data Engineering can then make use of geocoding and geoenrichment to further prepare our data for analysis.

***

## Step 1: Load and Clean Election Data

Goals:
- Handle missing values
- Correct truncated zeroes with FIPS field
- Restructure table format

To download and prepare the election data available, we will use a mix of Pandas, ArcPy, and the ArcGIS API for Python. Dataframes in Pandas serve as an effective way to format data and fix issues. First, let's import the necessary modules.

##### Import needed modules

In [2]:
# The import statements load each module
import arcgis
import pandas as pd
import os
import arcpy  # Best practice: Load arcpy last to maintain priority for namespace

##### Read data into Python

In [3]:
# Make reference to the file path for the csv, which should be in the same directory as the notebook
table_csv_path = "countypres_2000-2016.csv"

# Use Pandas to read the csv into a dataframe
data_df = pd.read_csv(table_csv_path, dtype={'year': str, 'FIPS': str})  # dtype parameter specifies that year and FIPS fields are string

# Use the head function to display the first five records of the dataframe
data_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001,President,Al Gore,democrat,4942.0,17208,20190722
1,2000,Alabama,AL,Autauga,1001,President,George W. Bush,republican,11993.0,17208,20190722
2,2000,Alabama,AL,Autauga,1001,President,Ralph Nader,green,160.0,17208,20190722
3,2000,Alabama,AL,Autauga,1001,President,Other,NaN,113.0,17208,20190722
4,2000,Alabama,AL,Baldwin,1003,President,Al Gore,democrat,13997.0,56480,20190722


### Handle missing data 

![Null Values](img/null_values.gif "Null Values")

In [4]:
# Set the field to check nulls for
field_to_check = "FIPS"

# Determine how many rows are in the table
rowcount = data_df.shape[0]

# Determine how many rows have null FIPS 
null_fips_rowcount = data_df.loc[data_df[field_to_check].isnull()].shape[0]

# Calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

# Use a print statement to report this information
print("There were "+str(null_fips_rowcount)+" records with null "+str(field_to_check)+" values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

There were 64 records with null FIPS values in the data.
This amounts to 0.13% of the available data.


In [6]:
# Use the notnull function and the loc function to create a new dataframe without null FIPS records
data_df = data_df.loc[data_df['FIPS'].notnull()]

### Explore and handle data types

![fix_truncated_zeroes](img/trunc_zeroes.gif "Fix Truncated Zeroes")

In [7]:
# Get the first five records of the table
data_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001,President,Al Gore,democrat,4942.0,17208,20190722
1,2000,Alabama,AL,Autauga,1001,President,George W. Bush,republican,11993.0,17208,20190722
2,2000,Alabama,AL,Autauga,1001,President,Ralph Nader,green,160.0,17208,20190722
3,2000,Alabama,AL,Autauga,1001,President,Other,NaN,113.0,17208,20190722
4,2000,Alabama,AL,Baldwin,1003,President,Al Gore,democrat,13997.0,56480,20190722


In [8]:
# Check how many records have a FIPS value with four characters
trunc_df = data_df.loc[data_df['FIPS'].str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / data_df.shape[0])*100

# Use another print statement (using the f format key) to report this information
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) has truncated FIPS values.")

10.36% of data (5228 rows) has truncated FIPS values.


Next, we need to fix the FIPS field since the data has leading zeroes truncated by its interpretation as a numeric field. We can create a simple function in python to determine if the value is four characters, and append a leading zero if that's the case. 

In [10]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(val)
    # Otherwise...
    else:
        # Return the value itself
        return str(val)

In [11]:
# Test helper function with truncated value
fix_trunc_zeros(7042)  # You should see an appended zero: "07042"

'07042'

In [12]:
# Run helper function on the FIPS field using the apply and lambda method 
data_df['FIPS'] = data_df['FIPS'].apply(lambda x: fix_trunc_zeros(x))

# Print information on the operation performed, and show the first few records to confirm it worked
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) had truncated FIPS IDs corrected.")
data_df.head()

10.36% of data (5228 rows) had truncated FIPS IDs corrected.


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,01001,President,Al Gore,democrat,4942.0,17208,20190722
1,2000,Alabama,AL,Autauga,01001,President,George W. Bush,republican,11993.0,17208,20190722
2,2000,Alabama,AL,Autauga,01001,President,Ralph Nader,green,160.0,17208,20190722
3,2000,Alabama,AL,Autauga,01001,President,Other,NaN,113.0,17208,20190722
4,2000,Alabama,AL,Baldwin,01003,President,Al Gore,democrat,13997.0,56480,20190722


### Reformat the table structure

![reformat_table](img/reformat_table.gif "Reformat Table")

We now need to reformat the structure of the table. Currently, each record corresponds to a candidate and their votes in a particular county. We need each record to correspond to each county, with fields showing the votes for different candidates for that election year. 

The animation above displays the reformatting necessary. It is possible to do this using Excel pivot tables, but a Python script to perform this might make things a bit easier to automate and share. The following code cell performs all the actions displayed above in the animation: 

In [13]:
# Reformat the dataframe by setting a multiindex (set_index with multiple fields) and pivoting the table (unstack)
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0


**Let's break this down step by step and fully understand it.**

First, take a few seconds to rewatch the animation above again and consider each step. 

In essence, we need to:

1. Set a few fields aside, "locking" them from the table pivot. 
2. Pivot the table using the remaining fields.
3. Give the pivoted fields designations for each party. 
4. Bring the locked fields back to our table. 

For context, Pandas has the following powerful capabilities that help us perform this operation: 

- The ability to set an index using multiple fields, which acts as our "locking" mechanism for step 1. 
- The ability to perform an operation using a "groupby" function, which lets us group each observation by candidate/party
- The ability to unstack a table, which lets us handle the table pivot.

**1. Setting an index using multiple fields** allows us to designate specific fields as index fields, which will not be impacted when a table pivot occurs. In other words, the fields specified in the set_index part of code above are "locked" when the later part of the code performs the table pivot via the "unstack" function.  

In [14]:
# Notice how running this cell uses the specified fields as row indices, 
# which prevents them from being "rotated" in the table pivot
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office'])

candidate       party  \
FIPS  year county   state   state_po office                                   
01001 2000 Autauga  Alabama AL       President          Al Gore    democrat   
                                     President   George W. Bush  republican   
                                     President      Ralph Nader       green   
                                     President            Other         NaN   
01003 2000 Baldwin  Alabama AL       President          Al Gore    democrat   
...                                                         ...         ...   
56043 2016 Washakie Wyoming WY       President     Donald Trump  republican   
                                     President            Other         NaN   
56045 2016 Weston   Wyoming WY       President  Hillary Clinton    democrat   
                                     President     Donald Trump  republican   
                                     President            Other         NaN   

                                                candidatevotes  totalvotes  \
FIPS  year county   state   state_po office                                  
01001 2000 Autauga  Alabama AL       President          4942.0       17208   
                                     President         11993.0       17208   
                                     President           160.0       17208   
                                     President           113.0       17208   
01003 2000 Baldwin  Alabama AL       President         13997.0       56480   
...                                                        ...         ...   
56043 2016 Washakie Wyoming WY       President          2911.0        3814   
                                     President           371.0        3814   
56045 2016 Weston   Wyoming WY       President           299.0        3526   
                                     President          3033.0        3526   
                                     President           194.0        3526   

                                                 version  
FIPS  year county   state   state_po office               
01001 2000 Autauga  Alabama AL       President  20190722  
                                     President  20190722  
                                     President  20190722  
                                     President  20190722  
01003 2000 Baldwin  Alabama AL       President  20190722  
...                                                  ...  
56043 2016 Washakie Wyoming WY       President  20190722  
                                     President  20190722  
56045 2016 Weston   Wyoming WY       President  20190722  
                                     President  20190722  
                                     President  20190722  

[50460 rows x 5 columns]

**2. The built-in groupby function** allows us to perform an operation using the unique values from a specified set of fields. This is useful because we can then count how many rows exist for a given FIPS and Year combination, which essentially lets us group data by the candidate that it pertains to. See the following example: 

In [15]:
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   data_df.groupby(['FIPS', 'year']).cumcount()+1])

candidate       party  \
FIPS  year county   state   state_po office                                     
01001 2000 Autauga  Alabama AL       President 1          Al Gore    democrat   
                                               2   George W. Bush  republican   
                                               3      Ralph Nader       green   
                                               4            Other         NaN   
01003 2000 Baldwin  Alabama AL       President 1          Al Gore    democrat   
...                                                           ...         ...   
56043 2016 Washakie Wyoming WY       President 2     Donald Trump  republican   
                                               3            Other         NaN   
56045 2016 Weston   Wyoming WY       President 1  Hillary Clinton    democrat   
                                               2     Donald Trump  republican   
                                               3            Other         NaN   

                                                  candidatevotes  totalvotes  \
FIPS  year county   state   state_po office                                    
01001 2000 Autauga  Alabama AL       President 1          4942.0       17208   
                                               2         11993.0       17208   
                                               3           160.0       17208   
                                               4           113.0       17208   
01003 2000 Baldwin  Alabama AL       President 1         13997.0       56480   
...                                                          ...         ...   
56043 2016 Washakie Wyoming WY       President 2          2911.0        3814   
                                               3           371.0        3814   
56045 2016 Weston   Wyoming WY       President 1           299.0        3526   
                                               2          3033.0        3526   
                                               3           194.0        3526   

                                                   version  
FIPS  year county   state   state_po office                 
01001 2000 Autauga  Alabama AL       President 1  20190722  
                                               2  20190722  
                                               3  20190722  
                                               4  20190722  
01003 2000 Baldwin  Alabama AL       President 1  20190722  
...                                                    ...  
56043 2016 Washakie Wyoming WY       President 2  20190722  
                                               3  20190722  
56045 2016 Weston   Wyoming WY       President 1  20190722  
                                               2  20190722  
                                               3  20190722  

[50460 rows x 5 columns]

**3. The ability to unstack a table** allows us to perform the table pivot, which essentially "rotates" the table and makes rows into columns (or columns into rows).

In [16]:
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

candidate  \
                                                             1   
FIPS  year county  state   state_po office                       
01001 2000 Autauga Alabama AL       President          Al Gore   
      2004 Autauga Alabama AL       President       John Kerry   
      2008 Autauga Alabama AL       President     Barack Obama   
      2012 Autauga Alabama AL       President     Barack Obama   
      2016 Autauga Alabama AL       President  Hillary Clinton   
...                                                        ...   
56045 2000 Weston  Wyoming WY       President          Al Gore   
      2004 Weston  Wyoming WY       President       John Kerry   
      2008 Weston  Wyoming WY       President     Barack Obama   
      2012 Weston  Wyoming WY       President     Barack Obama   
      2016 Weston  Wyoming WY       President  Hillary Clinton   

                                                                            \
                                                            2            3   
FIPS  year county  state   state_po office                                   
01001 2000 Autauga Alabama AL       President  George W. Bush  Ralph Nader   
      2004 Autauga Alabama AL       President  George W. Bush        Other   
      2008 Autauga Alabama AL       President     John McCain        Other   
      2012 Autauga Alabama AL       President     Mitt Romney        Other   
      2016 Autauga Alabama AL       President    Donald Trump        Other   
...                                                       ...          ...   
56045 2000 Weston  Wyoming WY       President  George W. Bush  Ralph Nader   
      2004 Weston  Wyoming WY       President  George W. Bush        Other   
      2008 Weston  Wyoming WY       President     John McCain        Other   
      2012 Weston  Wyoming WY       President     Mitt Romney        Other   
      2016 Weston  Wyoming WY       President    Donald Trump        Other   

                                                         party              \
                                                   4         1           2   
FIPS  year county  state   state_po office                                   
01001 2000 Autauga Alabama AL       President  Other  democrat  republican   
      2004 Autauga Alabama AL       President    NaN  democrat  republican   
      2008 Autauga Alabama AL       President    NaN  democrat  republican   
      2012 Autauga Alabama AL       President    NaN  democrat  republican   
      2016 Autauga Alabama AL       President    NaN  democrat  republican   
...                                              ...       ...         ...   
56045 2000 Weston  Wyoming WY       President  Other  democrat  republican   
      2004 Weston  Wyoming WY       President    NaN  democrat  republican   
      2008 Weston  Wyoming WY       President    NaN  democrat  republican   
      2012 Weston  Wyoming WY       President    NaN  democrat  republican   
      2016 Weston  Wyoming WY       President    NaN  democrat  republican   

                                                          candidatevotes  \
                                                   3    4              1   
FIPS  year county  state   state_po office                                 
01001 2000 Autauga Alabama AL       President  green  NaN         4942.0   
      2004 Autauga Alabama AL       President    NaN  NaN         4758.0   
      2008 Autauga Alabama AL       President    NaN  NaN         6093.0   
      2012 Autauga Alabama AL       President    NaN  NaN         6363.0   
      2016 Autauga Alabama AL       President    NaN  NaN         5936.0   
...                                              ...  ...            ...   
56045 2000 Weston  Wyoming WY       President  green  NaN          449.0   
      2004 Weston  Wyoming WY       President    NaN  NaN          580.0   
      2008 Weston  Wyoming WY       President    NaN  NaN          658.0   
      2012 Weston  Wyoming WY 

**4. Finally, we put it all together and rename the output columns**, using the pandas ability to rename fields and removing extraneous fields we no longer need. This produces the table format we need.

In [17]:
# Reformat the dataframe by setting a multiindex (set_index with multiple fields) and pivoting the table (unstack)
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total",
                                "state_po": "state_abbrev"
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]

# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Print out the first few records to confirm everything worked
df_out.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0


### Calculate Additional Columns/Features

##### Calculate total votes for non major party

In [ ]:
# Calculate votes that did not go for the Democrat or Republican party
df_out['votes_other'] = df_out['votes_total'] - (df_out['votes_dem'] + df_out['votes_gop'])
df_out.head()

##### Create additional attributes (voter percentages and raw differences)

In [ ]:
# Calculate voter share attributes
df_out['voter_share_major_party'] = (df_out['votes_dem'] + df_out['votes_gop']) / df_out['votes_total']
df_out['voter_share_dem'] = df_out['votes_dem'] / df_out['votes_total']
df_out['voter_share_gop'] = df_out['votes_gop'] / df_out['votes_total']
df_out['voter_share_other'] = df_out['votes_other'] / df_out['votes_total']

# Calculate raw difference attributes
df_out['rawdiff_dem_vs_gop'] = df_out['votes_dem'] - df_out['votes_gop']
df_out['rawdiff_gop_vs_dem'] = df_out['votes_gop'] - df_out['votes_dem']
df_out['rawdiff_dem_vs_other'] = df_out['votes_dem'] - df_out['votes_other']
df_out['rawdiff_gop_vs_other'] = df_out['votes_gop'] - df_out['votes_other']
df_out['rawdiff_other_vs_dem'] = df_out['votes_other'] - df_out['votes_dem']
df_out['rawdiff_other_vs_gop'] = df_out['votes_other'] - df_out['votes_gop']

# Calculate percent difference attributes
df_out['pctdiff_dem_vs_gop'] = (df_out['votes_dem'] - df_out['votes_gop']) / df_out['votes_total']
df_out['pctdiff_gop_vs_dem'] = (df_out['votes_gop'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_dem_vs_other'] = (df_out['votes_dem'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_gop_vs_other'] = (df_out['votes_gop'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_other_vs_dem'] = (df_out['votes_other'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_other_vs_gop'] = (df_out['votes_other'] - df_out['votes_gop']) / df_out['votes_total']

df_out.head()

***

## Step 2: Geoenable election data

Goals:
   - Retrieve 2016 USA county population GIS data
   - Perform join, bringing geometry to election data
   - Calculate Voter Turnout per county
   - Convert dataframe to feature class

We will now bring geometry data for each county into the table. 

### Retrieve 2016 USA county population GIS data

A useful source of data is the [ArcGIS Living Atlas of the World](https://livingatlas.arcgis.com), where we can find a service containing [Voting Age Population totals (citizens aged 18+) for each county](https://www.arcgis.com/home/item.html?id=2e8aaf91178c4c91b974d0bc4234dbfa). 

This dataset is included with this project to allow you to append the election data to county geometry and allow us to calculate voting turnout for each county. The next few cells will load this feature class into a spatially-enabled dataframe and allow us to geoenable the election data we have been preparing. Additionally, this feature class allows us to actually calculate voter turnout since it includes voting-age population counts for 2016. 

We will now reference this ArcGIS Pro project and its file geodatabase using ArcPy as well. Let's get started.

In [23]:
# Authenticate with a GIS using the ArcGIS API for Python
gis = arcgis.gis.GIS()

In [24]:
# Search for USA_Counties
search = gis.content.search("USA Counties owner:esri_dm", item_type="feature_service", outside_org=True, sort_field="numViews")
# Use the correct index to reference the search result
counties_item = search[1]
counties_item

<Item title:"USA Counties (Generalized)" type:Feature Layer Collection owner:esri_dm>

In [26]:
# Read the layer into a dataframe
counties_df = pd.DataFrame.spatial.from_layer(counties_item.layers[0])
counties_df = counties_df[['FIPS', 'NAME', 'STATE_FIPS', 'STATE_NAME', 'OBJECTID', 'POPULATION', 'POP_SQMI', 'SHAPE', 'Shape__Area', 'Shape__Length']]
counties_df.head()

,FIPS,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape__Area,Shape__Length
0,02013,Aleutians East,02,Alaska,1,3129,0.4,"{""rings"": [[[-17677357.9077045, 7719547.390490...",48862299112.445297,3299974.838988
1,38097,Traill,38,North Dakota,1201,8154,9.5,"{""rings"": [[[-10779852.8004753, 5981411.784141...",4881825510.71875,283774.464607
2,38099,Walsh,38,North Dakota,1202,11037,8.5,"{""rings"": [[[-10898454.0980662, 6139525.267360...",7683728314.69141,411795.184706
3,02016,Aleutians West,02,Alaska,2,5609,1.6,"{""rings"": [[[-18550860.9655057, 7080282.503091...",18314912687.242199,2455084.558028
4,38101,Ward,38,North Dakota,1203,75147,36.6,"{""rings"": [[[-11239339.6270714, 6082615.148233...",11921048654.257799,596478.705924


### Perform join, bringing geometry and population column to election data

We now have a dataframe with election data ('df_out') and a spatially-enabled dataframe of county voting-age population data ('counties_df'). Let's merge the datasets. 

In [28]:
# Join the data to our election data table
geo_df = pd.merge(df_out, counties_df, left_on='FIPS', right_on="FIPS", how='left')
# Visualize the merged data, notice the SHAPE column at the end
geo_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape__Area,Shape__Length
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",2188822787.74219,229109.164169
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",2188822787.74219,229109.164169
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",2188822787.74219,229109.164169
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",2188822787.74219,229109.164169
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",2188822787.74219,229109.164169


In [29]:
#Plot
geo_df.query('state_po == "NC" & year == "2016"').spatial.plot(
   renderer_type='c',              # for class breaks renderer
   method='esriClassifyQuantile',  # classification algorithm
   class_count=5,                  # choose the number of classes
   col='votes_dem',                # numeric column to classify
   cmap='viridis',                 # color map to pick colors from for each class
   alpha=0.3                       # specify opacity
)

MapView(layout=Layout(height='400px', width='100%'))

In [30]:
# Create a copy of the data, and perform a query
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

,FIPS,year,county,state,state_po,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape__Area,Shape__Length
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",2188822787.74219,229109.164169
9,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215.0,Baldwin,01,Alabama,1250,209227,127.6,"{'rings': [[[-9793119.56121114, 3654173.030111...",5832085136.22461,442546.872592
14,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469.0,Barbour,01,Alabama,1251,27072,29.9,"{'rings': [[[-9544908.60175304, 3714348.259590...",3245945815.84375,266874.305406
19,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819.0,Bibb,01,Alabama,1463,22932,36.6,"{'rings': [[[-9731533.81242838, 3896866.748799...",2330690457.46484,223825.974676
24,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588.0,Blount,01,Alabama,1966,57596,88.5,"{'rings': [[[-9681232.86521302, 4010079.855362...",2503707670.69922,249967.91142


### Convert dataframe to feature class

We can now finally convert our data to feature classes! The ArcGIS API for Python (which was invoked by using "import arcgis") lets us export the spatially-enabled dataframe to a feature class so we can do further analysis.

**Note: Executing the following cell may take a few minutes**

In [31]:
# Create a subset of the data just for the 2016 presidential election
fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\Data Engineering and Visualization\Data Engineering and Visualization.gdb"
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Feature Class Location: Dataset C:\Users\albe9057\Documents\ArcGIS\Projects\Data Engineering and Visualization\Data Engineering and Visualization.gdb does not exist or is not supported
Failed to execute (CreateFeatureclass).


Select the Map tab (titled "Data Engineering"), and view the added layers to the Table of Contents. You will now see counties across the United States with 2016 voting turnout data.

***

## Step 3: Geoenrich election data

Goals:
   - Use Geoenrichment to bring demographic and socioeconomic variables to use in analysis

Geoenrichment in ArcGIS Pro allows us to add columns of data for each county that can help us analyze relationships and potentially model voter turnout. Geoenrichment can be performed using ArcPy, but we recommend that you use the GeoProcessing Enrich tool to explore potential variables that may help explain voter turnout. You may now reference the MOOC guidance for the steps covering how to geoenrich this data. 

***